In [3]:
import pandas as pd
import numpy as np
from scipy import optimize

import os

def iones(nombre):
    nombre = str(nombre)
    path = os.getcwd() + '/templates/'
    dict_csv = { os.path.basename(x).split('.')[0]: path + '/' + x for x in os.listdir(path) if x.endswith('.csv') }
    ion_df = pd.read_csv(dict_csv[nombre]).fillna(0)

    indice = ion_df['GreenHow']

    copy = ion_df.copy()
    del copy['GreenHow']
    del copy['Ingredientes']

    A = copy.to_numpy()
    AT = A.transpose()
    
    return ion_df, indice, A, AT


def objetivo(nombre):
    path = os.getcwd() + '/inputs/'
    dict_csv = { os.path.basename(x).split('.')[0]: path + '/' + x for x in os.listdir(path) if x.endswith('.csv') }
    df = pd.read_csv(dict_csv[nombre]).fillna(0)

    target = df.iloc[0].to_numpy()[2:]
    const = df['VALOR'].to_numpy()[1:]

    return target, const

def corrida(indice, constraints, matriz):

    lista = []
    index = []

    for i in range(len(indice)):
        if constraints[i] == 1:
            print('Añadiendo: ' + indice[i])
            lista.append(matriz[i])
            index.append(indice[i])
        if constraints[i] == 0:
            print('No se utilizara : ' + indice[i])
    A = np.array(lista)

    AT = A.transpose()

    return A, AT, index

def loss(x0, k, A):
    lista = [sum((k - np.dot(x0, A))**2)/k.shape[0] for i in range(x0.shape[0])]
    
    return sum(lista)

def estefania(guardar=False):

    variedad = input('Ingrese el nombre de la variedad: ')

    df, indice, matriz, trans = iones()

    elementos = list(df.iloc[0][2:].index)

    target, const = objetivo(variedad)

    A, AT, nuevoindice = corrida(indice, const, matriz)

    x = np.random.rand(A.shape[0]) 

    print('X = ' , x)

    loss(x, target, A)

    res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
                        method='L-BFGS-B', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='nelder-mead', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='SLSQP', bounds=[(0,None)]*x.shape[0])

    #res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True, 'tol': 1e-10},
    #                    method='BFGS', bounds=[(0,None)]*x.shape[0])    


    # resultados
    d = pd.DataFrame({'fertilizante': nuevoindice, 'g/l': res.x})

    # % de error

    error = (target - np.dot(res.x, A))*100/1

    err = [round(num, 2) for num in error]

    e = pd.DataFrame({'error[%]': err})

    resultados = pd.DataFrame(elementos, columns=['elemento'])
    resultados = resultados.join(e)

    resultados = d.join(resultados)

    if guardar == True:

        current = os.getcwd()
        # guardar resultados
        resultados.to_csv(current + '/outputs/' + 'resultados' + variedad + '.csv', index=False)


    return resultados


</b>
Electronic<br>
Salt <br>
Target <br>
Estimator <br>
For <br>
Advanced <br>
Nutrition <br>
In <br>
Agriculture <br>

# ESTEFANIA

In [ ]:
estefania(guardar=True)

### paso a paso: 

## Cargamos los datos de las sales

In [4]:
df, indice, matriz, trans = iones('iones_2')

print('Las sales disponibles para el estudio son: ')
print(indice)

elementos = list(df.iloc[0][2:].index)


Las sales disponibles para el estudio son: 
0                    CALCIO BI SC 25 KG
1             calcio samurai con amonio
2                 MAP COLOSSAL SC 25 KG
3                 MKP COLOSSAL SC 25 KG
4     KERF ( Nitrato de Potasio mas Mg)
5                      NITAMKS SC 25 KG
6                   nitrato de magnesio
7                NITROKSUL NKS SC 25 KG
8                     SOLUCROS SC 25 KG
9                     sulfato de amonio
10                      SULMAG SC 50 KG
11                 ***Sulfato de calcio
12                     ***Quelato Ca95%
Name: GreenHow, dtype: object


## Cargamos la solución objetivo

In [8]:
variedad = 'kale-001'

target, const = objetivo(variedad)


for i in range(len(elementos)):
    print('El objetivo del ' + elementos[i] + ' es: ' +  str(target[i]) + ' g/l')

El objetivo del H+ es: 0.0 g/l
El objetivo del NO3- es: 9.38693625193657 g/l
El objetivo del NH4+ es: 4.17800052831859 g/l
El objetivo del H2PO4- es: 1.12998866589083 g/l
El objetivo del K+ es: 5.37221795855717 g/l
El objetivo del Ca+2 es: 3.74270173162333 g/l
El objetivo del Mg+2 es: 1.85147089076322 g/l
El objetivo del SO4-2 es: 2.1830656479027 g/l
El objetivo del Fe es: 0.01790670606142 g/l
El objetivo del Mn es: 0.0091008372770294 g/l
El objetivo del Zn es: 0.0023604947597016 g/l
El objetivo del Cu es: 0.0015736631731344 g/l
El objetivo del B es: 0.0462534690101758 g/l
El objetivo del Mo es: 0.0005211590577444 g/l
El objetivo del Co es: 0.0 g/l
El objetivo del Cl- es: 0.0 g/l
El objetivo del Na+ es: 0.0 g/l


## Hacemos la corrida con los fertilizantes seleccionados

In [9]:
A, AT, nuevoindice = corrida(indice, const, matriz)

No se utilizara : Ácido nítrico
No se utilizara : Ácido fosfórico
No se utilizara : Ácido sulfúrico
Añadiendo: Nitrato de amonio, Nitamks
Añadiendo: Nitrato de Potasio más Mg, KERF
Añadiendo: Nitrato de potasio más S, Nitroksul
Añadiendo: Nitrato de calcio samurai
Añadiendo: Nitrato de calcio bi
Añadiendo: Nitrato de magnesio
Añadiendo: Fosfato monoamónico, Map colossal
Añadiendo: Fosfato monopotásico, Mkp colossal
Añadiendo: Sulfato de amonio
Añadiendo: Sulfato de potasio, Solucros
Añadiendo: Sulfato de calcio
Añadiendo: Sulfato de magnesio, Sulmag
No se utilizara : Sulfato de fierro
No se utilizara : Sulfato de manganeso
No se utilizara : Sulfato de zinc
No se utilizara : Sulfato de cobre
No se utilizara : Quelato Ca 95%
Añadiendo: Quelato Fe 6%, microhow
Añadiendo: Quelato de Mn 13%, microhow
Añadiendo: Quelato de Zn 15%, microhow
Añadiendo: Quelato Cu 15%, microhow
No se utilizara : Quelato full mix
No se utilizara : Quelato full mix B
Añadiendo: Ácido bórico, abo
No se utilizara :

In [10]:
for i in range(len(nuevoindice)):
    print('El ' + nuevoindice[i] + ' es el # ' + str(i)  )


El Nitrato de amonio, Nitamks es el # 0
El Nitrato de Potasio más Mg, KERF es el # 1
El Nitrato de potasio más S, Nitroksul es el # 2
El Nitrato de calcio samurai es el # 3
El Nitrato de calcio bi es el # 4
El Nitrato de magnesio es el # 5
El Fosfato monoamónico, Map colossal es el # 6
El Fosfato monopotásico, Mkp colossal es el # 7
El Sulfato de amonio es el # 8
El Sulfato de potasio, Solucros es el # 9
El Sulfato de calcio es el # 10
El Sulfato de magnesio, Sulmag es el # 11
El Quelato Fe 6%, microhow es el # 12
El Quelato de Mn 13%, microhow es el # 13
El Quelato de Zn 15%, microhow es el # 14
El Quelato Cu 15%, microhow es el # 15
El Ácido bórico, abo es el # 16


## Creamos el vector a optimizar 

In [18]:
x = np.random.rand(A.shape[0])
print('Vector aleatoreo de longitud ' + str(len(x)) + ':', x)

Vector aleatoreo de longitud 17: [0.013038   0.62354595 0.65370701 0.17742372 0.57420564 0.51559192
 0.58282315 0.87481378 0.73531501 0.51972971 0.5713472  0.30976104
 0.12936303 0.80855374 0.30494876 0.36095278 0.12683083]


In [13]:
x[16] = 0.00013

x

array([3.69295890e-01, 5.90401619e-01, 5.94452075e-01, 4.89837447e-01,
       6.67110465e-01, 7.64108715e-01, 5.44171642e-01, 7.82221714e-02,
       4.95221696e-01, 1.57400979e-01, 9.58101961e-01, 4.13140542e-01,
       1.44079720e-01, 2.49834392e-01, 3.80774257e-01, 3.97588767e-01,
       1.30000000e-04])

# Función perdida 

In [19]:
print('A es de dimension: ', A.shape)
print('AT es de dimension: ', AT.shape)
print('target es de dimension: ', target.shape)
print('x es de dimension: ', x.shape)


A es de dimension:  (17, 17)
AT es de dimension:  (17, 17)
target es de dimension:  (17,)
x es de dimension:  (17,)


In [20]:
loss(x, target, A)

932.620746757508

In [21]:
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='SLSQP')
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='BFGS')
#res = optimize.minimize(loss, x0=x, args=(k, A), options={'disp': True}, method='Nelder-Mead')
res = optimize.minimize(loss, x0=x, args=(target, A), options={'disp': True}, 
                        method='L-BFGS-B', bounds=[(0,1)]*x.shape[0])

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           17     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.32621D+02    |proj g|=  8.74814D-01

At iterate    1    f=  1.57912D+02    |proj g|=  1.00000D+00

At iterate    2    f=  8.65941D+00    |proj g|=  9.51955D-01

At iterate    3    f=  4.18081D+00    |proj g|=  9.28591D-01

At iterate    4    f=  2.34736D+00    |proj g|=  9.57484D-01

At iterate    5    f=  2.22686D+00    |proj g|=  9.18324D-01

At iterate    6    f=  2.12342D+00    |proj g|=  8.85659D-01

At iterate    7    f=  2.08504D+00    |proj g|=  9.31705D-01

At iterate    8    f=  2.01586D+00    |proj g|=  9.28497D-01

At iterate    9    f=  1.83455D+00    |proj g|=  9.26556D-01

At iterate   10    f=  1.81655D+00    |proj g|=  9.16649D-01

At iterate   11    f=  1.74131D+00    |proj g|=  4.71840D-01

At iterate   12    f=  1.70343D+00    |proj g|=  4.32398D-01

At iterate   13    f=  1.6

In [22]:
# resultados
d = pd.DataFrame({'fertilizante': nuevoindice, 'g/l': res.x})

# % de error

error = (target - np.dot(res.x, A))*100/1

err = [round(num, 2) for num in error]

e = pd.DataFrame({'error[%]': err})

resultados = pd.DataFrame(elementos, columns=['elemento'])
resultados = resultados.join(e)

resultados = d.join(resultados)
resultados

,fertilizante,g/l,elemento,error[%]
0,"Nitrato de amonio, Nitamks",0.207099,H+,0.00
1,"Nitrato de Potasio más Mg, KERF",0.443132,NO3-,-30.04
2,"Nitrato de potasio más S, Nitroksul",0.000000,NH4+,28.06
3,Nitrato de calcio samurai,0.000000,H2PO4-,-27.97
4,Nitrato de calcio bi,0.409247,K+,23.57
5,Nitrato de magnesio,0.000000,Ca+2,60.94
6,"Fosfato monoamónico, Map colossal",0.165669,Mg+2,59.64
7,"Fosfato monopotásico, Mkp colossal",0.000000,SO4-2,-59.59
8,Sulfato de amonio,0.000000,Fe,0.00
9,"Sulfato de potasio, Solucros",0.000000,Mn,-0.00


## Gradiente descendente

In [410]:
print('A es de dimension: ', A.shape)
print('AT es de dimension: ', AT.shape)
print('k es de dimension: ', k.shape)
print('x es de dimension: ', x.shape)

A es de dimension:  (14, 7)
AT es de dimension:  (7, 14)
k es de dimension:  (7,)
x es de dimension:  (14,)


In [69]:
grad = lambda x: (-2/7)*(sum([np.dot((k[i] - np.dot(x, A[:,i])), A[:,i]) for i in range(A.shape[1])]))  

x = np.random.rand(A.shape[0])


print(x)
grad(x[1])


[0.48493831 0.82158739 0.92309745 0.32309576 0.85806003 0.52072956
 0.79377436 0.42070142 0.87617722 0.79958896 0.51304218 0.68733127
 0.16162787 0.34753273]


110.85150861358929

In [90]:
itermax = 5
iternum = 1
alfa = 0.01

x = np.random.rand(A.shape[0])


while True:

    xnuevo = np.ones(x.shape[0])
    print(xnuevo)
    for i in range(x.shape[0]):
        xnuevo[i] *=  x[i] - alfa*grad(x[i])
    print(xnuevo)
    x = xnuevo.copy()

    iternum += 1
    print('Iteracion #' + str(iternum))

    if iternum == itermax:
        print(xnuevo)
        break

    



[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 0.38803855  2.40041526  2.88434117  2.93456591 -0.98157776  2.78440451
 -0.57346545 -0.12360223 -0.18617435  1.15120954 -0.93201978 -0.23402997
 -0.79617713  2.45360293]
Iteracion #2
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.44611281 -6.59803364 -8.53244826 -8.73321343  6.92092982 -8.13296781
  5.28956766  3.49131309  3.74143488 -1.60453827  6.7228299   3.93272988
  6.17982115 -6.81064265]
Iteracion #3
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ -2.78336583  29.37179276  37.10429853  37.90682442 -24.66805056
  35.50744083 -18.14694738 -10.95871912 -11.95854003   9.41111246
 -23.87617857 -12.72321049 -21.70558996  30.22166246]
Iteracion #4
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[  14.12328266 -114.41169989 -145.32112585 -148.5290917   101.60372728
 -138.93794862   75.53668483   46.80291912   50.79953953  -34.62214802
   98.4383486    53.85618451   89.76177599 -117.80891489]
Iteracion #5
[  14.12328266 -114.41169989 -145.32112585

In [84]:
error = k - np.dot(x1, A)
for item in error:
    print(str(((item*100)/1).round(4)) + ' % de error para el elemento # ' + str(1 + error.tolist().index(item))) 

-2329.0681 % de error para el elemento # 1
-1184.4613 % de error para el elemento # 2
-832.3824 % de error para el elemento # 3
-1829.7247 % de error para el elemento # 4
-1094.4554 % de error para el elemento # 5
-298.0074 % de error para el elemento # 6
-1737.585 % de error para el elemento # 7
